In [1]:
import torch
import torch.nn as nn

# 定义多头自注意力类，继承自nn.Module
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, hidden_size, num_header, masked=False):
        super(MultiHeadSelfAttention, self).__init__()
        # 确保hidden_size可以被num_header整除
        assert hidden_size % num_header == 0, f"header的数目没办法整除:{hidden_size}, {num_header}"
        
        self.hidden_size = hidden_size  # 输入向量的维度
        self.num_header = num_header    # 注意力头的数量
        
        # 定义WQ、WK、WV，它们都是将输入向量映射到不同空间的线性层
        self.wq = nn.Sequential(
            nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size)
        )
        self.wk = nn.Sequential(
            nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size)
        )
        self.wv = nn.Sequential(
            nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size)
        )
        # 定义WO，用于在最后将多头的输出合并回原始维度
        self.wo = nn.Sequential(
            nn.Linear(in_features=self.hidden_size, out_features=self.hidden_size),
            nn.ReLU()  # 激活函数
        )
        self.masked = masked  # 是否使用掩码

    # 分割函数，将Q、K、V的输出分割成多个头
    def split(self, vs):
        n, t, e = vs.shape  # n: batch size, t: sequence length, e: feature dimension
        # 将vs重塑并交换维度，以适应多头机制
        vs = torch.reshape(vs, shape=(n, t, self.num_header, e // self.num_header))
        vs = torch.permute(vs, dims=(0, 2, 1, 3))
        return vs

    # 前向传播函数
    def forward(self, x):
        # 获取Q、K、V
        q = self.wq(x)  # 输入x经过WQ线性变换
        k = self.wk(x)  # 输入x经过WK线性变换
        v = self.wv(x)  # 输入x经过WV线性变换
        
        # 分割Q、K、V为多个头
        q = self.split(q)
        k = self.split(k)
        v = self.split(v)
        
        # 计算Q和K的相关性，得到注意力权重
        # torch.permute重新排列多维张量维度的函数。
        scores = torch.matmul(q, torch.permute(k, dims=(0, 1, 3, 2)))  # scores.shape == [n, h, t, t]
        
        # 如果使用掩码，则在上三角部分加上一个很大的负数，使得上三角的权重在Softmax后接近0
        if self.masked:
            mask = torch.ones((t, t))
            mask = torch.triu(mask, diagonal=1) * -10000  # 上三角为-10000
            mask = mask[None][None]  # 广播到[n, h, t, t]
            scores = scores + mask
        
        # 将分数转换为权重，使用Softmax函数
        alpha = torch.softmax(scores, dim=-1)  # alpha.shape == [n, h, t, t]
        
        # 使用权重和V相乘，得到加权的V
        v = torch.matmul(alpha, v)  # v.shape == [n, h, t, v]
        
        # 将多头的输出合并回原始维度
        v = torch.permute(v, dims=(0, 2, 1, 3))  # 交换维度
        n, t, _, _ = v.shape  # 重新计算维度
        v = torch.reshape(v, shape=(n, t, -1))  # 重塑为原始维度
        v = self.wo(v)  # 经过WO线性变换和ReLU激活函数
        return v

# 测试函数
def t0():
    # 创建一个简单的输入张量token_id
    token_id = torch.tensor([
        [1, 3, 5],
        [1, 6, 3],
        [2, 3, 1],
        [5, 1, 2],
        [6, 1, 2]
    ])
    
    # 创建一个简单的Embedding层
    emb_layer = nn.Embedding(num_embeddings=10, embedding_dim=8)
    x1 = emb_layer(token_id)  # 将token_id嵌入到8维空间
    
    # 打印一些输出
    print(x1[0][0])  # 第一个样本的第一个token的嵌入向量
    print(x1[1][0])  # 第二个样本的第一个token的嵌入向量
    print("=" * 100)
    
    # 创建多头自注意力实例
    att = MultiHeadSelfAttention(hidden_size=8, num_header=2)
    # 将嵌入向量通过多头自注意力层
    x3 = att(x1)
    # 打印一些输出
    print(x3[0][0])  # 第一个样本的第一个token经过自注意力后的向量
    print(x3[1][0])  # 第二个样本的第一个token经过自注意力后的向量
    print(x3[1][1])  # 第二个样本的第二个token经过自注意力后的向量

# 程序入口
if __name__ == '__main__':
    t0()

tensor([ 2.1515, -0.5459, -0.8552,  0.2976,  0.0460,  0.0296,  1.4862,  0.9704],
       grad_fn=<SelectBackward0>)
tensor([ 2.1515, -0.5459, -0.8552,  0.2976,  0.0460,  0.0296,  1.4862,  0.9704],
       grad_fn=<SelectBackward0>)
tensor([0.0806, 0.0000, 0.0000, 0.2075, 0.0000, 0.3678, 0.0000, 0.0000],
       grad_fn=<SelectBackward0>)
tensor([0.0821, 0.0438, 0.0000, 0.2411, 0.0000, 0.3375, 0.0000, 0.0000],
       grad_fn=<SelectBackward0>)
tensor([0.3642, 0.0816, 0.0000, 0.2423, 0.0000, 0.3399, 0.0000, 0.0000],
       grad_fn=<SelectBackward0>)
